In [1]:
import os

In [2]:
%pwd

'd:\\Text-Summarization-End-to-End-NLP-Projects\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text-Summarization-End-to-End-NLP-Projects'

In [5]:
# Importing the dataclass decorator from the dataclasses module.
# This decorator simplifies the creation of classes that are primarily used to store data.
from dataclasses import dataclass

# Importing the Path class from the pathlib module.
# Path is used for handling filesystem paths in a way that works across different operating systems.
from pathlib import Path

# Defining a data class for configuration settings related to data ingestion.
# The `frozen=True` argument makes the instances of this class immutable,
# meaning their attributes cannot be modified after creation.
@dataclass(frozen=True)
class DataIngestionConfig:
    # The root directory where all data-related operations will occur.
    root_dir: Path
    
    # The URL from which the data will be sourced.
    source_URL: str
    
    # The local path where the downloaded data file will be stored.
    local_data_file: Path
    
    # The directory where the downloaded data will be extracted/unzipped.
    unzip_dir: Path


In [12]:
# Importing all constants from the `textSummarizer.constants` module.
# This could include configuration settings, file paths, or other fixed values
# used across the text summarization project.
from src.textSummarizer.constants import *

# Importing utility functions from the `textSummarizer.utils.common` module.
# - `read_yaml`: A function to read and parse YAML files, likely used for loading configurations.
# - `create_directories`: A function to create directories on the filesystem,
#   ensuring required folder structures exist for the project.
from src.textSummarizer.utils.common import read_yaml, create_directories


In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-21 16:30:32,913: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-21 16:30:32,913: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-21 16:30:32,913: INFO: common: created directory at: artifacts]
[2024-11-21 16:30:32,913: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-21 16:30:48,165: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 54F9:6DA3F:B7E81C:BFB0B0:673F19FA
Accept-Ranges: bytes
Date: Thu, 21 Nov 2024 11:31:08 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990030-FJR
X-Cache: M